In [ ]:
from transformers import AutoTokenizer, AutoModel
from elasticsearch import Elasticsearch
import torch
from tqdm import tqdm

es = Elasticsearch("http://localhost:9200")
index_name = "index-e5large"
model_name = "intfloat/e5-large" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
new_mapping = {
    "properties": {
        "embedding": { 
            "type": "dense_vector",
            "dims": 1024,
            "index": True, 
            "similarity": "cosine" 
        }
    }
}

response = es.indices.put_mapping(
    index=index_name,
    body=new_mapping
)

print("Mapping updated:", response)

In [ ]:
def process_existing_data():

    def generate_embedding(text):
        text = "passage: " + text
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state[:, 0, :].cpu().numpy()[0].tolist()

    query = {"query": {"match_all": {}}}
    docs = es.search(index=index_name, body=query, size=10000)["hits"]["hits"]

    for doc in tqdm(docs, desc="Processing documents"):
        doc_id = doc["_id"]
        product_name = doc["_source"].get("productname")
        
        if product_name:
            try:
                embedding = generate_embedding(product_name)
                es.update(
                    index=index_name,
                    id=doc_id,
                    body={"doc": {"embedding": embedding}}
                )
            except Exception as e:
                print(f"Error processing doc {doc_id}: {str(e)}")

    print("Đã xử lý xong tất cả documents!")

if __name__ == "__main__":
    process_existing_data()

In [6]:
def full_text_search(query_text, es_client, index_name="index-e5large", top_k=10):

    search_body = {
        "size": top_k,
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["productname"],
                "type": "best_fields"
            }
        }
    }
    
    return es_client.search(index=index_name, body=search_body)

query_text = "cá bống"
response = full_text_search(query_text, es)

if response["hits"]["hits"]:
    print(f"Tìm thấy {len(response['hits']['hits'])} kết quả cho '{query_text}':(full-text search)\n")
    for i, hit in enumerate(response["hits"]["hits"], 1):
        print(f"#{i} | Score: {hit['_score']:.4f}")
        print(f"   Tên sản phẩm: {hit['_source'].get('productname', 'N/A')}")
else:
    print("Không tìm thấy kết quả phù hợp")

Tìm thấy 10 kết quả cho 'cá bống':(full-text search)

#1 | Score: 12.7677
   Tên sản phẩm: Cá bống mối XICHMAFOOD khay 500gr +/- 20gr
#2 | Score: 11.3045
   Tên sản phẩm: Cá Bống Đục Làm Sạch Tuấn Nguyễn khay 300g (+/-15g)
#3 | Score: 11.3045
   Tên sản phẩm: Cá bống tẩm ghép oval nướng cán HaiNamFoods khay 70g
#4 | Score: 10.1421
   Tên sản phẩm: Cá bống mối VAN khay 300gr ( mạ băng 25% giữ độ tươi ngon )
#5 | Score: 9.8060
   Tên sản phẩm: Bánh con cá Bống Bang Mochi Socola đậu đỏ Orion gói 145gr (5gói*29gr)
#6 | Score: 9.8060
   Tên sản phẩm: [ GIÁ RẺ VÔ ĐỊCH ] Cá bống mối biển nguyên con TPFood khay 300gr +/-20gr
#7 | Score: 9.8060
   Tên sản phẩm:  Bánh Orion Con Cá Bống Bang Mochi Socola Đậu Đỏ gói 145g (5P*29g)
#8 | Score: 9.8060
   Tên sản phẩm: [TẾT] Bánh Orion Con Cá Bống Bang Mochi Socola Đậu Đỏ gói 348g 12P
#9 | Score: 9.4915
   Tên sản phẩm: QT Bánh con cá Bống Bang Mochi Socola đậu đỏ Orion gói 145gr (5gói*29gr)
#10 | Score: 9.4915
   Tên sản phẩm: [COMBO 3 GÓI] Bánh Orio

In [13]:
def semantic_search(query_text, index_name="index-e5large", top_k=10):

        query_text = "query: " + query_text
        inputs = tokenizer(query_text, return_tensors="pt", truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        query_vector = outputs.last_hidden_state[:, 0, :].cpu().numpy()[0].tolist()
        
        search_body = {
            "size": top_k,
            "query": {
                "script_score": {
                    "query": {"match_all": {}},
                    "script": {
                        "source": """
                            if (!doc.containsKey('embedding') || doc['embedding'].size() == 0) {
                                return 0
                            }
                            return cosineSimilarity(params.query_vector, 'embedding') + 1.0
                        """,
                        "params": {"query_vector": query_vector}
                    }
                }
            }
        }
        
        response = es.search(index=index_name, body=search_body)
        
        if response["hits"]["hits"]:
            print(f"Tìm thấy {len(response['hits']['hits'])} kết quả cho '{query_text}':(semantic search)\n")
            for i, hit in enumerate(response["hits"]["hits"], 1):
                print(f"#{i} | Score: {hit['_score']:.4f}")
                print(f"   Tên: {hit['_source'].get('productname', 'N/A')}")
        else:
            print("Không tìm thấy kết quả")

if __name__ == "__main__":
    semantic_search("cá bống")

Tìm thấy 10 kết quả cho 'query: cá bống':(semantic search)

#1 | Score: 1.8578
   Tên: [CỰC RẺ] Bông Cải Xanh Nhập Khẩu 300 - 500g
#2 | Score: 1.8547
   Tên: Chà bông cá hồi Chà Bông Việt hũ 40g
#3 | Score: 1.8519
   Tên: Cá viên Ngon Ngon Chà Bông Việt gói 500g
#4 | Score: 1.8476
   Tên: [XẢ TỒN] Bông cải xanh Nhập Khẩu 300-500g
#5 | Score: 1.8476
   Tên: Khô gà cay lá chanh Chà Bông Việt gói 50g
#6 | Score: 1.8475
   Tên: Bông Cải Xanh Baby Đà Lạt 250g +/-10g
#7 | Score: 1.8459
   Tên: Khô gà cay mật ong Chà Bông Việt gói 35g
#8 | Score: 1.8438
   Tên: Bông Cải Xanh Nhập Khẩu 300 - 500g
#9 | Score: 1.8434
   Tên: Cá bống mối VAN khay 300gr ( mạ băng 25% giữ độ tươi ngon )
#10 | Score: 1.8394
   Tên: Bông tắm tròn hình hoa cao cấp siêu tạo bọt


In [ ]:
def hybrid_search(query_text, index_name="index-e5large", top_k=10):
    query_text = "query: " + query_text
    inputs = tokenizer(query_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    query_vector = outputs.last_hidden_state[:, 0, :].cpu().numpy()[0].tolist()
    
    try:
        search_body = {
            "size": top_k,
            "query": {
                "bool": {
                    "should": [
                        {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": """
                                        if (!doc.containsKey('embedding') || doc['embedding'].size() == 0) {
                                            return 0
                                        }
                                        return cosineSimilarity(params.query_vector, 'embedding') + 1.0
                                    """,
                                    "params": {"query_vector": query_vector},
                                },
                            }
                        },
                        {
                            "multi_match": {
                                "query": query_text,
                                "fields": ["productname"],
                                "fuzziness": "AUTO",
                                "boost": 0.004 
                            }
                        }
                    ],
                    "minimum_should_match": 1
                }
            }
        }

        response = es.search(index=index_name, body=search_body)
        
        if response["hits"]["hits"]:
            print(f"Tìm thấy {len(response['hits']['hits'])} kết quả cho '{query_text}':(hybird)\n")
            for i, hit in enumerate(response["hits"]["hits"], 1):
                print(f"#{i} | Score: {hit['_score']:.4f}")
                print(f"   Tên: {hit['_source'].get('productname', 'N/A')}")
        else:
            print("Không tìm thấy kết quả")

    except Exception as e:
        print(f"Lỗi khi tìm kiếm: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    hybrid_search("cá bống")

In [ ]:
def hybrid_search1(query_text, index_name="index-e5large", top_k=10):
    query_text = "query: " + query_text
    inputs = tokenizer(query_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    query_vector = outputs.last_hidden_state[:, 0, :].cpu().numpy()[0].tolist()
    
    try:
        search_body = {
            "size": top_k,
            "query": {
                "bool": {
                    "should": [
                        {
                            "script_score": {
                                "query": {"exists": {"field": "embedding"}},  
                                "script": {
                                    "source": """
                                        // Giảm độ phức tạp script
                                        double similarity = cosineSimilarity(params.query_vector, 'embedding');
                                        return similarity > 0 ? (similarity + 1.0) * 1.5 : 0
                                    """,
                                    "params": {"query_vector": query_vector}
                                },  
                                "boost": 1
                            }
                        },
                        {
                            "multi_match": {
                                "query": query_text.replace("query: ", ""),  
                                "fields": ["productname"],  
                                "type": "best_fields",  
                                "fuzziness": "1",  
                                "boost": 0.01 
                            }
                        }
                    ],
                    "minimum_should_match": 1
                }
            }
        }

        response = es.search(index=index_name, body=search_body)
        
        if response["hits"]["hits"]:
            print(f"Tìm thấy {len(response['hits']['hits'])} kết quả cho '{query_text}':(hybird)\n")
            for i, hit in enumerate(response["hits"]["hits"], 1):
                print(f"#{i} | Score: {hit['_score']:.4f}")
                print(f"   Tên: {hit['_source'].get('productname', 'N/A')}")
        else:
            print("Không tìm thấy kết quả")

    except Exception as e:
        print(f"Lỗi khi tìm kiếm: {str(e)}")
        import traceback
        traceback.print_exc()

# Test
if __name__ == "__main__":
    hybrid_search1("cá bống")

In [12]:
def hybrid_search_fixed(query_text, index_name="index-e5large", top_k=10):
    query_text = "query: " + query_text
    inputs = tokenizer(query_text, return_tensors="pt", truncation=True, max_length=256).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    query_vector = outputs.last_hidden_state[:, 0, :].cpu().numpy()[0].tolist()
    
    try:
        search_body = {
            "size": top_k,
            "timeout": "5s",
            "query": {
                "bool": {
                    "should": [
                        {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                                    "params": {"query_vector": query_vector}
                                },
                                "boost": 1
                            }
                        },
                        {
                            "match": {
                                "productname": {
                                    "query": query_text.replace("query: ", ""),
                                    "fuzziness": "1",
                                    "operator": "or",
                                    "boost": 0.001
                                }
                            }
                        }
                    ],
                    "filter": [
                        {"exists": {"field": "embedding"}}
                    ]
                }
            },
            "rescore": {
                "window_size": top_k * 3,
                "query": {
                    "rescore_query": {
                        "script_score": {
                            "query": {"match_all": {}},
                            "script": {
                                "source": "cosineSimilarity(params.query_vector, 'embedding')",
                                "params": {"query_vector": query_vector}
                            }
                        }
                    },
                    "query_weight": 0.4,
                    "rescore_query_weight": 0.6
                }
            }
        }

        response = es.options(request_timeout=10).search(
            index=index_name, 
            body=search_body
        )

        if response["hits"]["hits"]:
            print(f"Tìm thấy {len(response['hits']['hits'])} kết quả:\n")
            for i, hit in enumerate(response["hits"]["hits"], 1):
                print(f"#{i} | Score: {hit['_score']:.4f}")
                print(f"   Tên: {hit['_source'].get('productname', 'N/A')}")
        else:
            print("Không tìm thấy kết quả")

    except Exception as e:
        print(f"Lỗi khi tìm kiếm: {str(e)}")
        import traceback
        traceback.print_exc()
    
if __name__ == "__main__":
    hybrid_search_fixed("cá bống")

Tìm thấy 10 kết quả:

#1 | Score: 1.2591
   Tên: [CỰC RẺ] Bông Cải Xanh Nhập Khẩu 300 - 500g
#2 | Score: 1.2575
   Tên: Chà bông cá hồi Chà Bông Việt hũ 40g
#3 | Score: 1.2542
   Tên: Cá viên Ngon Ngon Chà Bông Việt gói 500g
#4 | Score: 1.2493
   Tên: Khô gà cay lá chanh Chà Bông Việt gói 50g
#5 | Score: 1.2489
   Tên: [XẢ TỒN] Bông cải xanh Nhập Khẩu 300-500g
#6 | Score: 1.2488
   Tên: Bông Cải Xanh Baby Đà Lạt 250g +/-10g
#7 | Score: 1.2471
   Tên: Khô gà cay mật ong Chà Bông Việt gói 35g
#8 | Score: 1.2469
   Tên: Cá bống mối VAN khay 300gr ( mạ băng 25% giữ độ tươi ngon )
#9 | Score: 1.2453
   Tên: Bông Cải Xanh Nhập Khẩu 300 - 500g
#10 | Score: 1.2406
   Tên: Bông tắm tròn hình hoa cao cấp siêu tạo bọt
